# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()


normalized_histos = list()

Welcome to JupyROOT 6.24/06


In [2]:
# %load python/drawingTools.py


In [58]:
# %load samples.py

samples = [
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v150A.70B', 'ref'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.80El', '12.3'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.80Fl', '12.3-tp'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.71Cl', '11.1'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.72Cl', '11.1-pf'),
    HistoFile('nugun_alleta_pu200_tprate', 'PU200', 'v151B.81A', '12.3-hgc'),
    
]


In [59]:
for hf in samples:
    hf.print_primitives()

<HistoFile ../plots/histos_nugun_alleta_pu200_rate_v150A.70B.root, ref>
--- HistoClass: 
['RateHistos']
--- TPs: 
['PFNFtkEleEE', 'PFNFtkEmEE']
--- TP selections:
{   'PFNFtkEleEE': [   'EGq3EtaBC',
                       'EGq3EtaBCD',
                       'EGq1EtaBC',
                       'EGq3',
                       'EGq1EtaBCD',
                       'EGq1'],
    'PFNFtkEmEE': [   'EGq3EtaBC',
                      'EGq1EtaBCD',
                      'EGq1EtaBC',
                      'EGq3',
                      'EGq1',
                      'EGq3EtaBCD']}
--- GEN selections:
{'PFNFtkEleEE': [None], 'PFNFtkEmEE': [None]}
<HistoFile ../plots/histos_nugun_alleta_pu200_rate_v151A.80El.root, 12.3>
--- HistoClass: 
['RateHistos']
--- TPs: 
['TkEmEB', 'TkEleEE', 'TkEmEE', 'TkEleEB']
--- TP selections:
{   'TkEleEB': ['EtaFLooseTkID', 'all', 'EtaF', 'LooseTkID'],
    'TkEleEE': [   'EGq1EtaBC',
                   'EGq1',
                   'EGq3EtaBCD',
                   'EGq3Eta

In [60]:
import python.collections as collections
import python.selections as selections
from cfg import *

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]






In [61]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [62]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.RateHistos)


--- RateHistos
# of plots: 36
# of primitives: 12
--- RateHistos
# of plots: 60
# of primitives: 20
--- RateHistos
# of plots: 60
# of primitives: 20
--- RateHistos
# of plots: 60
# of primitives: 20
--- RateHistos
# of plots: 60
# of primitives: 20
--- RateHistos
# of plots: 120
# of primitives: 40
CPU times: user 62.4 ms, sys: 7.82 ms, total: 70.3 ms
Wall time: 73.2 ms


In [63]:
hplot.data['sample'].unique()

array(['ref', '12.3', '12.3-tp', '11.1', '11.1-pf', '12.3-hgc'],
      dtype=object)

In [19]:
hplot.data['tp'].unique()

array(['PFNFtkEleEE', 'PFNFtkEmEE', 'TkEmEB', 'TkEleEE', 'TkEmEE',
       'TkEleEB', 'HGCCl3d'], dtype=object)

In [20]:
hplot.data[hplot.data.tp == 'tkEleEE'].tp_sel.unique()

array([], dtype=object)

In [21]:
# %load python/rate_utilities.py
import ROOT


def convertRateToGraph(histo_rate, params=None):
    """
    Converts a rate TH1F to a graph.
    Optionally applyes a linear scale mapping to the X axis.
    """
    
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


def cutAtRate(histo, rate):
    """
    Determines the X value (pt cut) corresponding to Y value (rate)
    """
    cut = None
    for ix in range(1, histo.GetNbinsX()+1):
        if histo.GetBinContent(ix) <= rate:
            cut = histo.GetXaxis().GetBinLowEdge(ix)
            break
    return cut


In [22]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE', 'TkEmEE'], ['EGq1'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [23]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE', 'TkEmEE'], ['EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [24]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE', 'TkEmEE'], ['EGq1EtaBCD'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [25]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEleEE'], ['EGq3EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=1000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [26]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEE','TkEleEE'], ['EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=1000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [27]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEmEB'], ['all'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=10, y_max=60000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [28]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEB','TkEleEB'], ['LooseTkID'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [29]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEB','TkEleEB'], ['LooseTkID'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




0.0 60.0
0.0 60.0
0.0 60.0


In [ ]:
--- TP selections:
{   'HGCCl3d': [   'EmEtaABC',
                   'EgBdtLE950EtaBCD',
                   'EgBdtHE900EtaBCD',
                   'EgBdtLE975',
                   'EtaBC',
                   'EmEtaBCD',
                   'EgBdtHE975EtaBC',
                   'Em',
                   'EgBdtHE900EtaABC',
                   'EtaABC',
                   'EtaBCD',
                   'EgBdtLE900',
                   'EgBdtLE975EtaABC',
                   'EgBdtHE950EtaBCD',
                   'EgBdtHE975',
                   'EgBdtHE950',
                   'EgBdtLE950',
                   'EmEtaBC',
                   'EgBdtHE995EtaBC',
                   'EgBdtLE975EtaBCD',
                   'EgBdtLE975EtaBC',
                   'EgBdtLE995EtaBC',
                   'EgBdtLE900EtaBCD',
                   'EgBdtHE995EtaABC',
                   'all',
                   'EgBdtLE995EtaABC',
                   'EgBdtHE975EtaABC',
                   'EgBdtHE950EtaBC',
                   'EgBdtLE995EtaBCD',
                   'EgBdtLE900EtaBC',
                   'EgBdtHE975EtaBCD',
                   'EgBdtHE995EtaBCD',
                   'EgBdtLE900EtaABC',
                   'EgBdtHE900',
                   'EgBdtLE995',
                   'EgBdtHE900EtaBC',
                   'EgBdtHE950EtaABC',
                   'EgBdtLE950EtaABC',
                   'EgBdtLE950EtaBC',
                   'EgBdtHE995']}

In [45]:
hplot.labels_dict['EgBdtLE995EtaBCD']

KeyError: 'EgBdtLE995EtaBCD'

In [69]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-hgc'], 'PU200', ['HGCCl3d'], ['EgBdtLE975EtaBC', 'EtaBC', 'EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000


In [70]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-tp', '12.3-hgc'], 'PU200', ['HGCCl3d', 'TkEmEE'], ['EmEtaBC', 'EgBdtLE975EtaBC', 'EgBdtLE995EtaBC', 'EtaBC', 'EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=80000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




## Online to offline scaling

In [30]:
# NOTE: this version of the scaling params is clearly obsolete. It is here just to keep track of how to 
# apply online to offline scaling
# version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format('v142A'))

In [ ]:
scaling_params

In [ ]:
# eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

In [ ]:
tps = hplot.data.tp.unique().tolist()

for sample in samples:
    for tp in tps:
        for tp_sel in ['EGq4EtaBC']:
            hsets, labels, text = hplot.get_histo(histos.RateHistos, sample.type, 'PU200', tp, tp_sel, None)
            print(hsets)
            if hsets is None:
                continue
            for his in hsets:
                his.normalize(31000)
                his.h_pt_graph(
                    'online', 
                    'rate [kHz]', 
                    convertRateToGraph)
                his.h_pt_graph(
                    'offline', 
                    'rate [kHz]', 
                    lambda x: convertRateToGraph(
                        x, 
                        eval(scaling_params[
                                (scaling_params['sample'] == 'ele-'+sample.type) &
                                (scaling_params.pu == 'PU200') & 
                                (scaling_params.tp == tp) & 
                                (scaling_params.tp_sel == tp_sel)].params.iloc[0])))

In [ ]:
for sample_scaling, sample_rate, tp, tp_sel, tp_scaling, tp_sel_scaling in [
    ('ele-V9', 'TDR', 'EG', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'EGBRL', 'EtaF', 'EGBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleELBRL', 'EtaF', 'TkEleELBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleEL', 'EGq5EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'Iso0p1EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'EtaFIso0p1', 'TkEleELBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'EgEE', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('dele-NM', 'NM', 'EgEB', 'EtaF', 'EGBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEB', 'EtaF', 'tkEleEB', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEE', 'EGq5EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'Iso0p1EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'EtaFIso0p1', 'tkEleEB', 'allEtaF'),

]:
    
        
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_rate, 'PU200', tp, tp_sel, None)
    if hsets is None:
        continue
    for his in hsets:
        his.normalize(31000)
        
        his.h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        his.h_pt_graph(
            'offline', 
            'rate [kHz]', 
            lambda x: convertRateToGraph(
                x, 
                eval(scaling_params[
                        (scaling_params['sample'] == sample_scaling) &
                        (scaling_params.pu == 'PU200') & 
                        (scaling_params.tp == tp_scaling) & 
                        (scaling_params.tp_sel == tp_sel_scaling)].params.iloc[0])))

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11', 'TDR'], 'PU200', ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], 'EmEtaBC', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


## Compute fixed rate WPs

In [91]:
rate_points = [20, 100, 1000]

In [92]:
def cutAtRate(histo, rate):
    """
    Determines the X value (pt cut) corresponding to Y value (rate)
    """
    cut = None
    for ix in range(0, histo.GetN()):
        if histo.GetY()[ix] <= rate:
            cut_low = histo.GetX()[ix]
#             rate_low = histo.GetY()[ix]
            cut_high = histo.GetX()[ix-1]
#             rate_high = histo.GetY()[ix-1]
            cut_middle = cut_high+(cut_low-cut_high)/2.
            rate_middle = histo.Eval(cut_middle)
#             print(f'rate_mid: {rate_middle} @ cut_mid: {cut_middle}')
            if rate_middle <= rate:
                cut = cut_high
            else:
                cut = cut_low
#             print(f'rate: {rate} @ pt: {cut}')
            break
    return cut


In [93]:
--- TP selections:
{   'TkEleEB': ['LooseTkID', 'EtaF', 'all', 'EtaFLooseTkID'],
    'TkEleEE': [   'EGq3EtaBC',
                   'EGq1EtaBCD',
                   'EGq3EtaBCD',
                   'EGq1',
                   'EGq1EtaBC',
                   'EGq3'],
    'TkEmEB': ['all', 'EtaFLooseTkID', 'LooseTkID', 'EtaF'],
    'TkEmEE': [   'EGq3EtaBCD',
                  'EGq3',
                  'EGq1EtaBCD',
                  'EGq3EtaBC',
                  'EGq1EtaBC',
                  'EGq1']}

SyntaxError: invalid syntax (2936113466.py, line 1)

In [109]:
import pprint as pp

hps = hplot.data

tps_and_sel = [('PFNFtkEmEE', 'EGq3EtaBC'),
               ('PFNFtkEleEE', 'EGq3EtaBC'),
               ('TkEmEE', 'EGq3EtaBC'),
               ('TkEleEE', 'EGq3EtaBC'),
               ('TkEleEB', 'LooseTkID'),
               ('TkEleEB', 'EtaFLooseTkID'),
               ('TkEmEB', 'all'),
               ('TkEmEB', 'EtaF'),
               ('HGCCl3d', 'EgBdtLE995EtaBC'),
               ('HGCCl3d', 'EgBdtLE975EtaBC')]
    
# smp = samples[0]
for smp in samples:
    wps = {}
    for tp,tp_sel in tps_and_sel:
        print(f'-- TP: {tp}')
        print(f'   sel: {tp_sel}')
        hsets, labels, text = hplot.get_histo(histos.RateHistos, smp.type, 'PU200', tp, tp_sel, None)
        if hsets is None:
            continue
        if tp not in wps.keys():
            wps[tp] = {}

        hsets[0].h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        rate_cuts = {}
        for rate in rate_points:
            ptcut = cutAtRate(hsets[0].g_pt_online, rate)
#                 ptcut_eval = hsets[0].g_pt_online.Eval(rate)
            rate_cuts[rate] = ptcut
            print(f'       rate: {rate}kHz @ pt: {ptcut}GeV')
            wps[tp][tp_sel] = rate_cuts

    pp.pprint(wps)

    import json
    with open('rate_pt_wps_{}.json'.format(smp.version), 'w') as fp:
        json.dump(wps, fp)

-- TP: PFNFtkEmEE
   sel: EGq3EtaBC
       rate: 20kHz @ pt: 32.0GeV
       rate: 100kHz @ pt: 20.0GeV
       rate: 1000kHz @ pt: 11.0GeV
-- TP: PFNFtkEleEE
   sel: EGq3EtaBC
       rate: 20kHz @ pt: 22.0GeV
       rate: 100kHz @ pt: 13.0GeV
       rate: 1000kHz @ pt: 99.0GeV
-- TP: TkEmEE
   sel: EGq3EtaBC
No match found for: sample: ref pu: PU200, tp: TkEmEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEE
   sel: EGq3EtaBC
No match found for: sample: ref pu: PU200, tp: TkEleEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEB
   sel: LooseTkID
No match found for: sample: ref pu: PU200, tp: TkEleEB, tp_sel: LooseTkID, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEB
   sel: EtaFLooseTkID
No match found for: sample: ref pu: PU200, tp: TkEleEB, tp_sel: EtaFLooseTkID, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEmEB
   sel: all
No 

In [95]:
dir(samples[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build_file_primitive_index',
 'build_file_primitive_index_oldStyle',
 'histo_file',
 'histo_filename',
 'label',
 'name',
 'oldStyle',
 'open_file',
 'print_file_primitive_index',
 'print_primitives',
 'type',
 'version']

In [105]:
import tabulate

table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if len(tp_wps) == 0:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]', 
    f'pT @ {rate_points[2]}kHz [GeV]']))
    
                    




sample    obj and sel.                pT @ 20kHz [GeV]    pT @ 100kHz [GeV]    pT @ 1000kHz [GeV]
--------  ------------------------  ------------------  -------------------  --------------------
ref       PFNFtkEmEE, EGq3EtaBC                     32                   20                    11
ref       PFNFtkEleEE, EGq3EtaBC                    22                   13                    99
12.3      TkEmEE, EGq3EtaBC                         32                   20                    11
12.3      TkEleEE, EGq3EtaBC                        22                   13                    99
12.3      TkEleEB, LooseTkID                        28                   17                     6
12.3      TkEleEB, EtaFLooseTkID                    28                   17                     6
12.3      TkEmEB, all                               49                   29                    15
12.3      TkEmEB, EtaF                              49                   29                    15
12.3-tp   TkEmEE, 

In [106]:
import tabulate

rate_points = [20, 100]
table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        if smp.type == 'ref':
            continue
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if not 'TkEleEE' in tp or "EtaFLoose" in tp_wps:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)

                #                 print(tp_sel)
#                 print(wps)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]']))
    
                    




sample    obj and sel.          pT @ 20kHz [GeV]    pT @ 100kHz [GeV]
--------  ------------------  ------------------  -------------------
12.3      TkEleEE, EGq3EtaBC                  22                   13
12.3-tp   TkEleEE, EGq3EtaBC                  22                   13
11.1      TkEleEE, EGq3EtaBC                  22                   13
11.1-pf   TkEleEE, EGq3EtaBC                  22                   13


In [108]:
import tabulate

rate_points = [20, 100, 1000]
table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        if not 'hgc' in smp.type:
            continue
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if not 'HGCCl3d' in tp:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)

                #                 print(tp_sel)
#                 print(wps)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]']))
    
                    




          sample                      obj and sel.    pT @ 20kHz [GeV]    pT @ 100kHz [GeV]
--------  ------------------------  --------------  ------------------  -------------------
12.3-hgc  HGCCl3d, EgBdtLE995EtaBC              36                  23                   14
12.3-hgc  HGCCl3d, EgBdtLE975EtaBC              31                  20                   12


In [79]:
import tabulate
tp = 'TkEleEE'
iso_vars = []
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


NameError: name 'iso_pt_wps' is not defined

In [45]:
import tabulate
tp = 'PFNFtkEmEB'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


NameError: name 'iso_pt_wps' is not defined

In [ ]:
import pprint as pp

iso_pt_wps = {}

histo_primitives = hplot.data

tp_sels = [
    ('PFNFtkEmEB', histo_primitives[histo_primitives.tp == 'PFNFtkEmEB'].tp_sel.unique().tolist()),
    ('PFNFtkEmEE', [sel for sel in histo_primitives[histo_primitives.tp == 'PFNFtkEmEE'].tp_sel.unique().tolist() if 'EtaBC' not in sel]),   
]
    
# print (tp_sels)

for tp, tp_sel_list in tp_sels:
    iso_pt_wps[tp] = {}
    for sel in tp_sel_list:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, ['ref'], 'PU200', tp, sel, None)
        
        for his in hsets:
            his.normalize(31000)
        rate_cuts = {}
        for rate in rate_points:
            rate_cuts[rate] = cutAtRate(hsets[0].h_pt, rate)
        iso_pt_wps[tp][sel] = rate_cuts

pp.pprint(iso_pt_wps)

import json
with open('iso_pt_wps.json', 'w') as fp:
    json.dump(iso_pt_wps, fp)

In [ ]:
import tabulate
tp = 'PFNFtkEmEB'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


In [ ]:
import tabulate
tp = 'PFNFtkEmEE'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'EGq5{iso_var}WP{iso_wp}Pt{pt_wp}EtaABC'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['EGq5EtaABC'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')
